In [58]:
# read the data
#imports"
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import Draw

# Create a Path object for the current directory
current_directory = Path.cwd()
print("Current Directory:", current_directory.resolve())

# Creating a Path object for an example file that does not yet exist
example_file_path = current_directory / "database.csv"

# Reading the contents of the file to test if we have well acess to our data
if example_file_path.exists():
    with example_file_path.open("r") as file:
        first_line = file.readline()
        print(first_line)
else:
    print("The file does not exist.")

Current Directory: C:\Users\33783\Documents\EPFL\Etudes\BA4\Amazing-Project
Reactant 1;Reactant 2;Product 1;Product 2;Product 3;ea



In [59]:
# Create the Data Frame
from rdkit import Chem
from rdkit.Chem import PandasTools
import pandas as pd
from pathlib import Path
import os

dataFrame= pd.read_csv("database.csv")
print (dataFrame)

     Reactant 1;Reactant 2;Product 1;Product 2;Product 3;ea
0     [Br-:2];[Br:1][C:5]([C:4]([H:3])([C:31]#[N:32]...    
1     [Br-:2];[Br:1][C:5]([C:4]([H:3])([C:31]#[N:32]...    
2     [Br-:2];[Br:1][C:5]([C:4]([H:3])([H:31])[C:41]...    
3     [Br-:2];[Br:1][C:5]([C:4]([H:3])([H:31])[H:41]...    
4     [Br-:2];[Br:1][C:5]([C:4]([H:3])([H:31])[H:41]...    
...                                                 ...    
3620  [F:1][C@:5]([C@:4]([H:3])([N:31]([H:32])[H:33]...    
3621  [F:1][C@:5]([C@:4]([H:3])([N:31]([H:32])[H:33]...    
3622  [F:1][C@:5]([C@:4]([H:3])([N:31]([H:32])[H:33]...    
3623  [F:1][C@:5]([C@:4]([H:3])([N:31]([H:32])[H:33]...    
3624  [F:1][C@:5]([C@:4]([H:3])([N:31]([H:32])[H:33]...    

[3625 rows x 1 columns]


In [10]:
# fonction to remove atome mapping numbers from smiles
import re

def remove_atom_mapping(smiles):
    """
    Remove atom mapping numbers from a SMILES-like notation.
    Args:
    - smiles (str): The SMILES-like notation with atom mapping numbers.
    Returns:
    - smiles_without_mapping (str): The SMILES notation without atom mapping numbers.
    """
    # Define a regular expression pattern to match atom mapping numbers
    pattern = r':\d+'
    smiles_without_mapping = re.sub(pattern, '', smiles)
    return smiles_without_mapping

In [11]:
# give smiles
def Smiles(smiles):
    """
    Remove atom mapping numbers, brackets, 'H' characters, and empty parentheses from a SMILES-like notation.
    Args:
    - smiles (str): The SMILES-like notation with atom mapping numbers, brackets, 'H' characters, and empty parentheses.
    Returns:
    - smiles_cleaned (str): The cleaned SMILES notation.
    """
    # Define a regular expression pattern to match atom mapping numbers, brackets, 'H' characters, and empty parentheses
    pattern = r'[:\[\]\ H]'
    # Use re.sub() to remove the specified characters
    smiles_cleaned = re.sub(pattern, '', smiles)
    # Remove empty parentheses only
    smiles_cleaned = re.sub(r'\(\)', '', smiles_cleaned)
    return smiles_cleaned

def convert_X(smiles):
    """
    Convert "X-" elements where X can be "Br", "Cl", or "F" into "[X-]" in a SMILES-like notation.
    Args:
    - smiles (str): The SMILES-like notation.
    Returns:
    - cleaned_smiles (str): The cleaned SMILES notation.
    """
    # Define the elements you want to convert
    elements = ['Br', 'Cl', 'F']
    # Iterate over each element
    for element in elements:
        # Replace "X-" with "[X-]"
        smiles = smiles.replace(element + '-', '[' + element + '-]')
    return smiles


In [12]:
# remove mapping from the Data Frame
for column in dataFrame.columns:
    # Apply the remove_atom_mapping function to each column
    dataFrame[column] = dataFrame[column].apply(remove_atom_mapping)
    dataFrame[column] = dataFrame[column].apply(Smiles)
    dataFrame[column] = dataFrame[column].apply(convert_X)
print(dataFrame)

     Reactant 1;Reactant 2;Product 1;Product 2;Product 3
0            [Br-];BrC(C(C#N)C#N);[Br-];BrC(C(C#N)C#N); 
1          [Br-];BrC(C(C#N))(C)C;[Br-];BrC(C(C#N))(C)C; 
2              [Br-];BrC(CC#N)(C)C;[Br-];BrC(CC#N)(C)C; 
3                      [Br-];BrC(C)(C);[Br-];BrC(C)(C); 
4                          [Br-];BrC(C)C;[Br-];BrC(C)C; 
...                                                 ... 
3620        FC@(C@(N)N+(O-)O)N;-;[F-];C(C@(N)N+(O-)O)N; 
3621  FC@(C@(N)N+(O-)O)(N+(O-)O)C;-;C(=C(/N+(O-)O)C)... 
3622  FC@(C@(N)N+(O-)O)(N+(O-)O);-;[F-];C(C@(N)N+(O-... 
3623  FC@(C@(N)N+(O-)O)(N)C#N;-;[F-];C@@(C@(N)N+(O-)... 
3624    FC@(C@(N)N+(O-)O)(N);-;[F-];C(C@(N)N+(O-)O)(N); 

[3625 rows x 1 columns]


In [13]:
# Split the column values at each ";" and expand them into separate columns
split_dataFrame = dataFrame['Reactant 1;Reactant 2;Product 1;Product 2;Product 3'].str.split(';', expand=True)

# Rename the columns
split_dataFrame.columns = ['Reactant 1', 'Reactant 2', 'Product 1', 'Product 2', 'Product 3']

# Display the resulting DataFrame
print(split_dataFrame)

                       Reactant 1       Reactant 2  \
0                           [Br-]   BrC(C(C#N)C#N)   
1                           [Br-]  BrC(C(C#N))(C)C   
2                           [Br-]    BrC(CC#N)(C)C   
3                           [Br-]        BrC(C)(C)   
4                           [Br-]          BrC(C)C   
...                           ...              ...   
3620           FC@(C@(N)N+(O-)O)N                -   
3621  FC@(C@(N)N+(O-)O)(N+(O-)O)C                -   
3622   FC@(C@(N)N+(O-)O)(N+(O-)O)                -   
3623      FC@(C@(N)N+(O-)O)(N)C#N                -   
3624         FC@(C@(N)N+(O-)O)(N)                -   

                        Product 1                 Product 2 Product 3  
0                           [Br-]            BrC(C(C#N)C#N)            
1                           [Br-]           BrC(C(C#N))(C)C            
2                           [Br-]             BrC(CC#N)(C)C            
3                           [Br-]                 BrC(C)(C)    

In [14]:
# Assuming your DataFrame is named dataFrame
# Access the third column and get its first element
element = split_dataFrame.iloc[0, 2]

# Print the element and its type
print("First element of the third column:", element)
print("Type of the element:", type(element))

First element of the third column: [Br-]
Type of the element: <class 'str'>


In [45]:
# Prompt the user to enter the molecule that he wants to form
print ("you can enter the name of the molecule or its SMILES, the racemic configuration is available. Only halogen molecules are taken in account in the data base")
string_input_mol = input("Enter the molecule that you want to form: ")

you can enter the name of the molecule or its SMILES, the racemic configuration is available. Only halogen molecules are taken in account in the data base


Enter the molecule that you want to form:  bromide


In [46]:
# function to determine if the user enters the SMILES notation or the usual name of the molecule
from rdkit import Chem

def is_smiles(smiles):
    """
    Check if a string represents a valid SMILES notation.
    Args:
    - smiles (str): The string to check.
    Returns:
    - is_valid (bool): True if the string is a valid SMILES notation, False otherwise.
    """
    if Chem.MolFromSmiles(smiles)== None:
        return False
    else:
        return True

# Check if the entered string is a valid SMILES notation
if is_smiles(string_input_mol):
    print(f"'{string_input_mol}' is a valid SMILES notation.")
else:
    print(f"'{string_input_mol}' is not a valid SMILES notation.")

'bromide' is not a valid SMILES notation.


[16:19:57] SMILES Parse Error: syntax error while parsing: bromide
[16:19:57] SMILES Parse Error: Failed parsing SMILES 'bromide' for input: 'bromide'


In [47]:
!pip install pubchempy

In [48]:
# function to convert name into SMILES

import pubchempy as pcp

def name_to_smiles(molecule_name):
    """
    Convert a molecule name to a SMILES notation using PubChemPy's PubChem database.
    Args:
    - molecule_name (str): The name of the molecule.
    Returns:
    - smiles (str): The SMILES notation of the molecule, or None if retrieval fails.
    """
    try:
        compound = pcp.get_compounds(molecule_name, 'name')
        if compound:
            return compound[0].canonical_smiles
        else:
            print("Error: Unable to retrieve molecule information. Please try with the SMILE of the molecule")
            return None
    except:
        print("Error: Unable to retrieve molecule information. Please try again")
        return None

In [49]:
# Call the function to convert the molecule name to SMILES notation
if is_smiles(string_input_mol)== False:
    string_input_mol = name_to_smiles(string_input_mol)
    if string_input_mol:
        print(f"SMILES notation for {string_input_mol}: {string_input_mol}")
else:
    print (string_input_mol)

[16:20:06] SMILES Parse Error: syntax error while parsing: bromide
[16:20:06] SMILES Parse Error: Failed parsing SMILES 'bromide' for input: 'bromide'


SMILES notation for [Br-]: [Br-]


In [50]:
# function to find the required molecule in the data Frame

def compare_molecule_with_data(element):
    return element == string_input_mol

# Iterate over the last three columns of the DataFrame
for column_name in split_dataFrame.columns[-3:]:
    column = split_dataFrame[column_name]
    # Check if any element in the column matches the input molecule
    if any(column.apply(compare_molecule_with_data)):
        print("Input molecule found in the DataFrame:")
        print(split_dataFrame[column.apply(compare_molecule_with_data)])
        print("Row number:", index)
        break
else:
    print("The product is not in the database")

Input molecule found in the DataFrame:
                        Reactant 1       Reactant 2 Product 1  \
0                            [Br-]   BrC(C(C#N)C#N)     [Br-]   
1                            [Br-]  BrC(C(C#N))(C)C     [Br-]   
2                            [Br-]    BrC(CC#N)(C)C     [Br-]   
3                            [Br-]        BrC(C)(C)     [Br-]   
4                            [Br-]          BrC(C)C     [Br-]   
...                            ...              ...       ...   
1293           BrC@(C@(N)N+(O-)O)N                -     [Br-]   
1294   BrC@(C@(N)N+(O-)O)(N+(O-)O)             [F-]     [Br-]   
1295   BrC@(C@(N)N+(O-)O)(N+(O-)O)                -     [Br-]   
1296  BrC@(C@(N)N+(O-)O)(N+(O-)O)N             [F-]     [Br-]   
1297        BrC@(C@(N)N+(O-)O)(N)C             [F-]     [Br-]   

                         Product 2 Product 3  
0                   BrC(C(C#N)C#N)            
1                  BrC(C(C#N))(C)C            
2                    BrC(CC#N)(C)C     

In [57]:
# Function that returns the row of the elements formation reaction
def compare_molecule_with_data(element):
    return element == string_input_mol

# Initialize a variable to store the row number
found_row_number = None

# Iterate over the last three columns of the DataFrame
for column_name in split_dataFrame.columns[-3:]:
    column = split_dataFrame[column_name]
    # Check if any element in the column matches the input molecule
    for index, value in column.items():
        if compare_molecule_with_data(value):
            # Store the row number where the molecule is found
            found_row_number = index
            print("Input molecule found in the DataFrame at row number:", found_row_number)
    else:
        continue  # Continue to the next column if molecule not found in current column

# Check if the molecule was found and return the row number
if found_row_number is not None:
   pass
else:
    print("The product is not in the database")


Input molecule found in the DataFrame at row number: 0
Input molecule found in the DataFrame at row number: 1
Input molecule found in the DataFrame at row number: 2
Input molecule found in the DataFrame at row number: 3
Input molecule found in the DataFrame at row number: 4
Input molecule found in the DataFrame at row number: 5
Input molecule found in the DataFrame at row number: 6
Input molecule found in the DataFrame at row number: 7
Input molecule found in the DataFrame at row number: 8
Input molecule found in the DataFrame at row number: 9
Input molecule found in the DataFrame at row number: 10
Input molecule found in the DataFrame at row number: 11
Input molecule found in the DataFrame at row number: 12
Input molecule found in the DataFrame at row number: 13
Input molecule found in the DataFrame at row number: 14
Input molecule found in the DataFrame at row number: 15
Input molecule found in the DataFrame at row number: 16
Input molecule found in the DataFrame at row number: 17
In

In [ ]:
# print the yield

In [32]:
# print the reaction
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem



ValueError: ChemicalReactionParserException: a reaction requires at least two > characters